In [6]:
# Library Import
# datasets 와 동일한 계층 폴더에서 실행하면 됨
import os
import json
import shutil
from tqdm import tqdm

In [7]:
def convert_coco_to_yolo_format(root_dir: str, json_file: str, save_dir: str):
# Check directory
    try:
        assert os.path.exists(os.path.join(root_dir, save_dir, "images")) == True
        assert os.path.exists(os.path.join(root_dir, save_dir, "labels")) == True
    except:
        os.makedirs(os.path.join(root_dir, save_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, save_dir, "labels"), exist_ok=True)
    finally:
        print("Finish make directory")

# Load json
    with open(os.path.join(root_dir, json_file), 'r') as f:
        coco_json = json.load(f)
    anoots = coco_json["annotations"]

    print("Start converting...")
    for image in tqdm(sorted(coco_json["images"], key=lambda x: x["id"])):
        w, h, file_name, image_id = image["width"], image["height"], image["file_name"], image["id"]
        file_name = file_name.split("/")[1]
    # filtering annotations
        obj_candits = list(filter(lambda x: x["image_id"] == image_id, anoots))
    # Save txt format to train yolo
        with open(os.path.join(root_dir, save_dir, "labels", f"{file_name[:-4]}.txt"), "w") as f:
            for obj_candit in obj_candits:
             # x1 y1 w h -> cx cy w h               
                cat_id = obj_candit["category_id"]

                x1, y1, width, height = obj_candit["bbox"]
                scaled_cx, scaled_cy = (x1+width/2) / w, (y1+height/2) / h
                scaled_width, scaled_height = width / w, height / h
                f.write("%s %.3f %.3f %.3f %.3f\n" %(cat_id, scaled_cx, scaled_cy, scaled_width, scaled_height))
            f.close()    # Copy image to new directory
        shutil.copy(os.path.join(root_dir, save_dir.split('_')[-1], file_name), os.path.join(root_dir, save_dir, "images", file_name))
    print("Finish converting...")

images, label folder생성

In [8]:
convert_coco_to_yolo_format("../../dataset", "train.json", "yolo_train")

Finish make directory
Start converting...


100%|██████████| 4883/4883 [00:14<00:00, 333.92it/s]

Finish converting...


In [10]:
convert_coco_to_yolo_format("../../dataset", "test.json", "yolo_test")

Finish make directory
Start converting...


  0%|          | 0/4871 [00:00<?, ?it/s]

100%|██████████| 4871/4871 [00:04<00:00, 1040.96it/s]

Finish converting...


images, label .txt로 valid 분할

In [11]:
from glob import glob
test_img_list = glob('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/images/*.jpg')
print(len(test_img_list))

4883


In [12]:
from sklearn.model_selection import train_test_split
train_list, valid_list = train_test_split(test_img_list, test_size=0.2, random_state = 2000)
print(len(train_list), len(valid_list))

3906 977


In [13]:
with open('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt', 'w') as f:
    f.write('\n'.join(train_list) + '\n')

with open('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt', 'w') as f:
    f.write('\n'.join(valid_list) + '\n')

In [14]:
# yaml파일 수정, 최초 1회만 실행해도 됨.
import yaml

with open('./yolov7/data/custom_data.yaml','r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt'
data['val'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt'
data['test'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images'

with open('./yolov7/data/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'download': 'bash ./scripts/get_coco.sh', 'names': ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'], 'nc': 10, 'test': './coco/test-dev2017.txt', 'train': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt', 'val': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt'}
{'download': 'bash ./scripts/get_coco.sh', 'names': ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'], 'nc': 10, 'test': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images', 'train': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt', 'val': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt'}
